In [1]:
import nltk
import re
import string
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import ssl
import pandas as pd 
import numpy
import matplotlib.pyplot as plt
import seaborn as sns
from langdetect import detect
from textblob import TextBlob

ssl._create_default_https_context = ssl._create_unverified_context
sid = SentimentIntensityAnalyzer()
nltk.download('vader_lexicon')

from sklearn.feature_extraction import text

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/natasha/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [2]:
tweets = pd.read_csv('../data/tweets.csv',delimiter=';',lineterminator='\n') 
#Context Scrapped from twitters from 2016-01-01 to 2019-03-29, Collecting Tweets containing Bitcoin or BTC Tools used: - Twint - Tweepy
#https://www.kaggle.com/datasets/alaix14/bitcoin-tweets-20160101-to-20190329 
tweets.head() #going to do sentiment analysis on the text data

/var/folders/tg/8mpzp58s7wd9h25zt2bffqd00000gn/T/ipykernel_19131/575731240.py:1: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  tweets = pd.read_csv('../data/tweets.csv',delimiter=';',lineterminator='\n')


,id,user,fullname,url,timestamp,replies,likes,retweets,text\r
0,1.132977e+18,KamdemAbdiel,Abdiel kamdem,NaN,2019-05-27 11:49:14+00,0,0,0,È appena uscito un nuovo video! LES CRYPTOMONN...
1,1.132977e+18,bitcointe,Bitcointe,NaN,2019-05-27 11:49:18+00,0,0,0,Cardano: Digitize Currencies; EOS https://t.co...
2,1.132977e+18,3eyedbran,Bran - 3 Eyed Raven,NaN,2019-05-27 11:49:06+00,0,2,1,Another Test tweet that wasn't caught in the s...
3,1.132977e+18,DetroitCrypto,J. Scardina,NaN,2019-05-27 11:49:22+00,0,0,0,Current Crypto Prices! \n\nBTC: $8721.99 USD\n...
4,1.132977e+18,mmursaleen72,Muhammad Mursaleen,NaN,2019-05-27 11:49:23+00,0,0,0,Spiv (Nosar Baz): BITCOIN Is An Asset &amp; NO...


After years of price fluctuations ranging between $100 and $900, bitcoin finally broke $1,000 again in January 2017. This kicked off a euphoric bull run phase. Prices doubled to $2,000 in mid-May and then skyrocketed to over $19,000 by December.1

In [3]:
#changing datetimeformat
tweets['timestamp'] = pd.to_datetime(tweets['timestamp'])
tweets = tweets.sort_values(by='timestamp', ascending=True)

In [4]:
#taking those from 2016-2018
tw_df = tweets[(tweets['timestamp'] >= '2016-01-01') & (tweets['timestamp'] <= '2017-12-31')]
tw_df.shape

(938074, 9)

In [5]:
#cleaning up the column names
tw_df.rename(columns={'text\r':'text'},inplace=True)
#drop duplicates based on user and text
tw_df = tw_df.drop_duplicates(subset=['user', 'text']).reset_index()
#taking only timestamp and text
tw_df = tw_df[['timestamp','text']]

tw_df.head()

/var/folders/tg/8mpzp58s7wd9h25zt2bffqd00000gn/T/ipykernel_19131/2831295270.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tw_df.rename(columns={'text\r':'text'},inplace=True)


,timestamp,text
0,2016-01-01 00:00:04+00:00,Current price: 431.05$ $BTCUSD $btc #bitcoin 2...
1,2016-01-01 00:00:05+00:00,Current price: 396.05€ $BTCEUR $btc #bitcoin 2...
2,2016-01-01 00:00:05+00:00,Current price: 291.77£ $BTCGBP $btc #bitcoin 2...
3,2016-01-01 00:00:53+00:00,#RDD / #BTC on the exchanges:\nCryptsy: 0.0000...
4,2016-01-01 00:02:32+00:00,#RDD / #BTC on the exchanges: Cryptsy: 0.00000...


In [6]:
tw_df.shape

(899513, 2)

In [7]:
tw_df.info() #there are almost 16M tweets regarding 'btc' or 'bitcoin' from 1st January 2016 to 29th March 2019 but will take from 2016-2018 instead

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 899513 entries, 0 to 899512
Data columns (total 2 columns):
 #   Column     Non-Null Count   Dtype              
---  ------     --------------   -----              
 0   timestamp  899513 non-null  datetime64[ns, UTC]
 1   text       899513 non-null  object             
dtypes: datetime64[ns, UTC](1), object(1)
memory usage: 13.7+ MB


In [8]:
#removal of non english word
def english_text(df):
    try: 
        return detect(df)
    except:
        return 'Other'

In [9]:
english_text(tw_df['text'][10])

'en'

In [10]:
tw_df['lang']=tw_df['text'].apply(lambda x: english_text(x))
tw_df.head()

,timestamp,text,lang
0,2016-01-01 00:00:04+00:00,Current price: 431.05$ $BTCUSD $btc #bitcoin 2...,en
1,2016-01-01 00:00:05+00:00,Current price: 396.05€ $BTCEUR $btc #bitcoin 2...,en
2,2016-01-01 00:00:05+00:00,Current price: 291.77£ $BTCGBP $btc #bitcoin 2...,en
3,2016-01-01 00:00:53+00:00,#RDD / #BTC on the exchanges:\nCryptsy: 0.0000...,en
4,2016-01-01 00:02:32+00:00,#RDD / #BTC on the exchanges: Cryptsy: 0.00000...,en


In [11]:
tw_df = tw_df[tw_df['lang']=='en']
tw_df.shape

(728893, 3)

In [12]:
tw_df.to_csv('../data/2-tweets_en_nodup.csv')